# Parsing
2025 labs done in Codespaces/Github

In [1]:
%pip install beautifulsoup4 requests pandas lxml openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

для себя:

https://docs.python.org/3/library/concurrent.futures.html

## UTILITIES

In [3]:
BASE_URL = 'http://magnitogorsk-citystar.ru'
FLATS_LIST_URL = BASE_URL + '/realty/prodazha-kvartir/'

def fetch_page(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'lxml')
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def get_flats_list_pages():
    pages = []
    for i in range(1, 22):
        url = f'{FLATS_LIST_URL}?p={i}'
        soup = fetch_page(url)
        if soup:
            pages.append(soup)
            print(f"Fetched page {i}")
        time.sleep(0.5)  # pause
    return pages

def extract_flats_from_page(soup):
    flats_data = []
    
    table = soup.find('table', {'id': 'tbl_10043'})
    if not table:
        print("No table found on page")
        return flats_data
    
    for tr in table.find_all('tr'):
        tds = tr.find_all('td', recursive=False)
        if len(tds) < 5:  # skip empty
            continue
        
        try:
            flat_id = int(tds[0].input['value'])
            link = tds[4].a['href']
            flats_data.append((flat_id, link))
        except (ValueError, TypeError, KeyError, IndexError) as e:
            print(f"Error parsing row: {e}")
            continue
    
    return flats_data

def extract_flat_details(flat_id, flat_link):
    url = BASE_URL + flat_link
    soup = fetch_page(url)
    
    if not soup:
        return None
    
    details = {}
    div = soup.find('div', 'adv-main-data')
    
    if div:
        # key-value pairs
        keys = [key.get_text(strip=True) for key in div.find_all('td', 'field-title')]
        values = [value.get_text(strip=True).replace('\xa0', ' ') for value in div.find_all('td', 'field')]
        
        # dict
        for key, value in zip(keys, values):
            details[key] = value
        
        # description
        note = div.find('td', 'note')
        if note:
            details['Описание'] = note.get_text(strip=True)
    
    # link id
    details['Ссылка'] = url
    details['ID'] = flat_id
    
    return details


## PARSING

In [4]:
def main_parsing():
    print("start lab parsing")
    
    # listing pages
    pages = get_flats_list_pages()
    
    # all flats data extract
    all_flats_data = []
    for page in pages:
        flats_data = extract_flats_from_page(page)
        all_flats_data.extend(flats_data)
    
    print(f"Found {len(all_flats_data)} flats")
    
    # parallel processing
    all_details = []
    
    # ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for flat_id, flat_link in all_flats_data:
            futures.append(executor.submit(extract_flat_details, flat_id, flat_link))
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                all_details.append(result)
                print(f"Processed {len(all_details)}/{len(all_flats_data)} flats")
            time.sleep(0.3)  # pause
    
    # df
    df = pd.DataFrame(all_details)
    
    # id as index
    if 'ID' in df.columns:
        df = df.set_index('ID')
    
    # autosave
    #df.to_csv('magnitogorsk_parsing.csv', encoding='utf-8-sig')
    #df.to_excel('magnitogorsk_parsing.xlsx')
    
    print('done')

    return df

In [5]:
df = main_parsing()

start lab parsing


Fetched page 1
Fetched page 2
Fetched page 3
Fetched page 4
Fetched page 5
Fetched page 6
Fetched page 7
Fetched page 8
Fetched page 9
Fetched page 10
Fetched page 11
Fetched page 12
Fetched page 13
Fetched page 14
Fetched page 15
Fetched page 16
Fetched page 17
Fetched page 18
Fetched page 19
Fetched page 20
Fetched page 21
No table found on page
No table found on page
No table found on page
No table found on page
Found 486 flats
Processed 1/486 flats
Processed 2/486 flats
Processed 3/486 flats
Processed 4/486 flats
Processed 5/486 flats
Processed 6/486 flats
Processed 7/486 flats
Processed 8/486 flats
Processed 9/486 flats
Processed 10/486 flats
Processed 11/486 flats
Processed 12/486 flats
Processed 13/486 flats
Processed 14/486 flats
Processed 15/486 flats
Processed 16/486 flats
Processed 17/486 flats
Processed 18/486 flats
Processed 19/486 flats
Processed 20/486 flats
Processed 21/486 flats
Processed 22/486 flats
Processed 23/486 flats
Processed 24/486 flats
Processed 25/486 flats

In [7]:
df.head(3)

,Цена,Этаж,Площадь,Кол-во комнат,Этажность дома,Описание,Ссылка,Адрес,Район,Состояние квартиры,...,"Районная поликлиника, школы, детские сады, магазины и т. д.",Год последнего ремонта,Окон на Север,Окон на Юг,Окон на Запад,Окон на Восток,Доступный транспорт,Ближайший паркинг,Микрорайон или ориентир,Правоустанавливающие документы
ID,,,,,,,,,,,,,,,,,,,,,
23812767,1 560 000р.(65 000р./м2)Подать заявку на ипотеку,1/2,"общая24 м2,жилая18 м2,кухни5 м2",Однокомнатная,2,id:47636. \nУстали от городского шума и суеты?...,http://magnitogorsk-citystar.ru/realty/prodazh...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23812765,2 770 000р.(63 532р./м2)Подать заявку на ипотеку,1/5,"общая43,6 м2,жилая30 м2,кухни6 м2",Двухкомнатная,5,id:47626. \nПредставляем вашему вниманию уютну...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Советская, 145/1",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23812768,2 080 000р.(65 000р./м2)Подать заявку на ипотеку,1/2,"общая32 м2,жилая20 м2,кухни12 м2",Однокомнатная,2,id:47637. \nУстали от городского шума и суеты?...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Ленина пр-т, 214",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


только важное, как вариант только первые столбцы

In [8]:
new_df = df.iloc[:, :8].copy()
new_df

,Цена,Этаж,Площадь,Кол-во комнат,Этажность дома,Описание,Ссылка,Адрес
ID,,,,,,,,
23812767,1 560 000р.(65 000р./м2)Подать заявку на ипотеку,1/2,"общая24 м2,жилая18 м2,кухни5 м2",Однокомнатная,2,id:47636. \nУстали от городского шума и суеты?...,http://magnitogorsk-citystar.ru/realty/prodazh...,NaN
23812765,2 770 000р.(63 532р./м2)Подать заявку на ипотеку,1/5,"общая43,6 м2,жилая30 м2,кухни6 м2",Двухкомнатная,5,id:47626. \nПредставляем вашему вниманию уютну...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Советская, 145/1"
23812768,2 080 000р.(65 000р./м2)Подать заявку на ипотеку,1/2,"общая32 м2,жилая20 м2,кухни12 м2",Однокомнатная,2,id:47637. \nУстали от городского шума и суеты?...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Ленина пр-т, 214"
23812769,2 080 000р.(63 609р./м2)Подать заявку на ипотеку,1/1,"общая32,7 м2,жилая30 м2,кухни0 м2",NaN,1,id:47638. \nПредставьте себе уединённый уголок...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Ленина пр-т, 212б"
23812766,1 250 000р.(30 488р./м2)Подать заявку на ипотеку,2/2,"общая41 м2,жилая20 м2,кухни7 м2",Однокомнатная,2,id:47633. \nПродам уютную 1-комнатную квартиру...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Садовая, 18"
...,...,...,...,...,...,...,...,...
15406262,8 000 000р.(76 190р./м2)Подать заявку на ипотеку,4/9,"общая105 м2,жилая83 м2,кухни10 м2",Четырехкомнатная,9,Предлагаю Вашему вниманию четырёхкомнатную ква...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Карла Маркса, 185"
16262155,5 500 000р.(84 615р./м2)Подать заявку на ипотеку,1/10,"общая65 м2,жилая43 м2,кухни12 м2",Двухкомнатная,10,"<p>В продаже уютная 2к квартира, в одном из лу...",http://magnitogorsk-citystar.ru/realty/prodazh...,"Жукова, 19/1"
16022577,5 560 000р.(76 374р./м2)Подать заявку на ипотеку,6/6,"общая72,8 м2,жилая44 м2,кухни8 м2",Трехкомнатная,6,<p>Для ценителей Ленинского района в продаже у...,http://magnitogorsk-citystar.ru/realty/prodazh...,"Строителей, 57"


In [ ]:
#df.to_csv('lab_parsing.csv', index=False)
#df.to_excel('lab_parsing.csv', index=False)

In [14]:
clean_df = new_df.replace({'\u2028': ' ', '\u2029': ' '}, regex=True)
clean_df.to_csv('lab_parsing.csv', index=False, lineterminator='\n')

Вывод по лабе: получен lab_parsing.csv файл парсингом